In [49]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [50]:
# Diretório onde estão armazenadas as imagens inteiras
image_dir = r'C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\dataset'

In [51]:
def load_image(image_path, target_size=(224, 224)):
    try:
        image = Image.open(image_path)
        image = image.resize(target_size)
        return np.array(image) / 255.0  # Normaliza a imagem para a faixa [0, 1]
    except Exception as e:
        print(f"Erro ao carregar a imagem {image_path}: {e}")
        return None

In [52]:
# Carregar IDs do grupo de treinamento de um arquivo de texto
train_ids_file = r'C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\training_lists\lower_resolution_train.txt'
with open(train_ids_file, 'r') as file:
    train_ids = [str(id) for id in json.loads(file.read())]

In [53]:
# Carregar IDs do grupo de validação de um arquivo de texto
val_ids_file = r'C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\training_lists\lower_resolution_val.txt'
with open(val_ids_file, 'r') as file:
    val_ids = [str(id) for id in json.loads(file.read())]

In [54]:
# Verificar se os IDs foram carregados corretamente
print(f"IDs de treinamento carregados: {train_ids[:5]} ... ({len(train_ids)} no total)")
print(f"IDs de validação carregados: {val_ids[:5]} ... ({len(val_ids)} no total)")

IDs de treinamento carregados: ['16078', '16831', '16073', '16072', '16830'] ... (3319 no total)
IDs de validação carregados: ['14175', '14176', '15540', '14172', '14678'] ... (1107 no total)


In [55]:
# Definir o número de classes
num_classes = 5

# Função para mapear IDs para rótulos automaticamente
def map_id_to_label(image_id):
    # Exemplo de mapeamento: hash do ID mod num_classes
    return int(image_id) % num_classes

In [56]:
# Função para carregar e redimensionar a imagem
def load_image(image_path, target_size=(224, 224)):
    try:
        image = Image.open(image_path)
        image = image.resize(target_size)
        return np.array(image) / 255.0  # Normaliza a imagem para a faixa [0, 1]
    except Exception as e:
        print(f"Erro ao carregar a imagem {image_path}: {e}")
        return None

In [57]:
# Função para carregar as imagens e as labels
def load_data(image_id, image_dir, target_size=(224, 224)):
    image_path = os.path.join(image_dir, f'{image_id}.jpg')
    if not os.path.exists(image_path):
        return None, None
    
    image = load_image(image_path, target_size)
    if image is None:
        return None, None

    label = map_id_to_label(image_id)
    return image, label

In [58]:
def data_generator(ids, image_dir, target_size=(224, 224)):
    for image_id in ids:
        image, label = load_data(image_id, image_dir, target_size)
        if image is not None and label is not None:
            yield image, label

In [59]:
def preprocess(image, label):
    image = tf.image.per_image_standardization(image)
    label = tf.cast(label, dtype=tf.int32)
    label = tf.one_hot(label, depth=num_classes)
    return image, label

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_ids, image_dir),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(val_ids, image_dir),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

In [60]:
# Aplicar preprocessamento
train_dataset = train_dataset.map(preprocess).batch(32).shuffle(buffer_size=len(train_ids))
val_dataset = val_dataset.map(preprocess).batch(32)

In [61]:
# Verificar o output do data generator
for image, label in data_generator(train_ids[:5], image_dir):
    print(f"Imagem shape: {image.shape}, Label: {label}")

Imagem shape: (224, 224, 3), Label: 3
Imagem shape: (224, 224, 3), Label: 1
Imagem shape: (224, 224, 3), Label: 3
Imagem shape: (224, 224, 3), Label: 2
Imagem shape: (224, 224, 3), Label: 0


In [62]:
#for image, label in train_dataset.take(1):
#    print(f'Imagem shape após preprocessamento: {image.numpy().shape}')
#    print(f'Label após preprocessamento: {label.numpy()}')

In [67]:
# Carregar o modelo InceptionV3 pré-treinado, excluindo a última camada
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models, optimizers

base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

In [68]:
# Escolher até onde descongelar as camadas
fine_tune_at = 75  # número de camadas a serem descongeladas

# Congelar todas as camadas até o fine_tune_at
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [69]:
# Adicionar camadas densas no topo do modelo base
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    
    # Adicionar camadas densas extras com Dropout
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # Dropout de 50% para ajudar a prevenir overfitting
    
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # Dropout de 50% para ajudar a prevenir overfitting
    
    layers.Dense(num_classes, activation='softmax')
])

In [70]:
# Ajustar o otimizador Adam com uma taxa de aprendizado menor
optimizer = optimizers.Adam(learning_rate=0.0001)  # Reduzir a taxa de aprendizado

In [71]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [72]:
# Definição de callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='model_checkpoint.h5', 
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        verbose=1
    )
]

In [ ]:
# Verificar algumas imagens e rótulos do conjunto de treinamento
#for image_batch, label_batch in train_dataset.take(1):
#    for i in range(5):  # Exibir 5 imagens do lote
#        image = image_batch[i]  # Obtém uma imagem do lote
#        label = label_batch[i]  # Obtém o respectivo rótulo do lote
#        # Reescala a imagem para o intervalo [0, 1] para a visualização
#        image_rescaled = (image.numpy() - image.numpy().min()) / (image.numpy().max() - image.numpy().min())
#        plt.imshow(image_rescaled)  # Exibe a imagem reescalada
#        plt.title(f'Label: {np.argmax(label.numpy())}')  # Corrigido para mostrar a classe correta
#        plt.show()

In [73]:
# Treinamento do modelo
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
    104/Unknown - 381s 1s/step - loss: 1.8784 - accuracy: 0.1914
Epoch 1: val_loss improved from inf to 1.60997, saving model to model_checkpoint.h5
104/104 [==============================] - 508s 2s/step - loss: 1.8784 - accuracy: 0.1914 - val_loss: 1.6100 - val_accuracy: 0.2034
Epoch 2/10
104/104 [==============================] - ETA: 0s - loss: 1.6107 - accuracy: 0.2074
Epoch 2: val_loss did not improve from 1.60997
104/104 [==============================] - 504s 2s/step - loss: 1.6107 - accuracy: 0.2074 - val_loss: 1.6108 - val_accuracy: 0.2025
Epoch 3/10
104/104 [==============================] - ETA: 0s - loss: 1.6086 - accuracy: 0.2101
Epoch 3: val_loss did not improve from 1.60997
104/104 [==============================] - 476s 2s/step - loss: 1.6086 - accuracy: 0.2101 - val_loss: 1.6137 - val_accuracy: 0.1872
Epoch 4/10
104/104 [==============================] - ETA: 0s - loss: 1.6115 - accuracy: 0.2058
Epoch 4: val_loss improved from 1.60997 to 1.60995, saving mod